# LAS Data Dive Group F

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import re
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Read in data

In [ ]:
LEAVERS = pd.read_csv("data/Workforce Data/LAS_Leavers_070319.csv")
STAFF = pd.read_csv("data/Workforce Data/LAS_Staff_in_Post_070319.csv")
STATIONS = pd.read_csv("data/Workforce Data/workplace location table.csv")


#From https://datashare.is.ed.ac.uk/handle/10283/2597
PC_SECTOR = gpd.read_file("data/postcode/Sectors.shp", crs={'init': 'epsg:4326'})

In [ ]:
STAFF["LengthOfService(Years)"] = pd.to_numeric(STAFF["LengthOfService(Years)"], float)

STAFF["LengthOfService(Months)"] = pd.to_numeric(STAFF["LengthOfService(Months)"], float)

STAFF["LengthOfServiceFLOAT"] = STAFF["LengthOfService(Years)"] + STAFF["LengthOfService(Months)"]/12

## Adding geospatial info to current staff table

In [ ]:
#Add station postcodes to staff data
STAFFM2 = pd.merge(STAFF, STATIONS, left_on="Location", right_on="esrLocationFull")
STAFFM2 = STAFFM2.rename(columns={"Location" : "WorkLocation", "postcode" : "PostcodeWork"})
STAFFM2

In [ ]:
#Function to extract coordinates
def extract_coords(point):
    lat = re.findall(r'[-]*[0-9]+[.]*[0-9]+', str(point))
    long = re.findall(r'\s([0-9]+[.]*[0-9]+)', str(point))
    return str(long[0] + "," + lat[0])

In [ ]:
#Merge staff data with postcode sector boundaries
STAFF_PC_AGG_GEO = pd.merge(STAFF, PC_SECTOR, left_on="PostcodeSector", right_on="name")

#Create geodatamframe
STAFF_PC_AGG_GEO = gpd.GeoDataFrame(STAFF_PC_AGG_GEO, geometry='geometry', crs={'init': 'epsg:4326'})

#Get centroids
STAFF_PC_AGG_GEO["centroids"] = STAFF_PC_AGG_GEO.centroid

STAFF_PC_AGG_GEO["centroids_str"] = STAFF_PC_AGG_GEO["centroids"].apply(extract_coords)

In [ ]:
#Get postcodes of departments
INFERR = pd.DataFrame(STAFFM2.groupby(by=["Department", "WorkLocation", "PostcodeWork"]).count()["IDnumber"].sort_values(ascending = False))
INFERR.columns = ["Count"]

INFERR = INFERR.reset_index()

INFERR2 = INFERR.set_index(["Department", "WorkLocation"])

#Input this in to online postcode search website https://gridreferencefinder.com/postcodeBatchConverter/
INFERR2.to_csv("Org Department postcodes.csv")

In [ ]:
#Import this data
Workplace_coords = pd.read_csv("station_pc_latlong.csv")
Workplace_coords

In [ ]:
#Add co-ordinates to staff data
MERGED_MEGA = pd.merge(STAFFM2, Workplace_coords,  left_on="PostcodeWork", right_on="Postcode", how="left")

#Export to csv for use in Tableau
MERGED_MEGA.to_csv("OD COUNTS5.csv")

## Calculating turnover

In [ ]:
STAFF["LatestStartDate"] = pd.to_datetime(STAFFM2["LatestStartDate"])
LEAVERS["Employee Latest Start Date"] = pd.to_datetime(LEAVERS["Employee Latest Start Date"])
LEAVERS["Termination Date"] = pd.to_datetime(LEAVERS["Termination Date"])

In [ ]:
Leavers_2018 = len(LEAVERS.loc[LEAVERS["Leaving year"] == 2018])

Staff_Dec_2018 = len(STAFF.loc[STAFF["LatestStartDate"] < pd.to_datetime("2018-12-31")])
Staff_Jan_2018 = len(STAFF.loc[STAFF["LatestStartDate"] < pd.to_datetime("2018-01-01")])
Average_staff2018 = (Staff_Dec_2018 + Staff_Jan_2018)/2 + Leavers_2018/2
Average_staff2018

Turnover_Rate = Leavers_2018/Average_staff2018
Turnover_Rate

In [ ]:
LEAVERS3 = LEAVERS[["Employee Number", "Employee Latest Start Date", "Termination Date", "Org L5"]]

STAFF["Termination Date"] = np.nan
STAFFM3 = STAFF[["IDnumber", "LatestStartDate", "Directorate", "Termination Date"]]

In [ ]:
STAFFM3.columns = ["Employee Number", "Employee Latest Start Date", "Org L5", "Termination Date"]

In [ ]:
MERGED_STAFF = pd.concat([LEAVERS3, STAFFM3], axis=0, ignore_index=True)

In [ ]:
Leave2018 = MERGED_STAFF.loc[MERGED_STAFF["Termination Date"] < pd.to_datetime("2018-12-31")]
Leave2018 = Leave2018.loc[Leave2018["Termination Date"] > pd.to_datetime("2018-01-01")]